<a href="https://colab.research.google.com/github/BodrulJalal/CS676-Algorithm-of-Data-Science/blob/main/Project_2_Deliverable_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Quiet install of the Tinytroupe Repo
!pip install -q git+https://github.com/microsoft/TinyTroupe.git@main > /dev/null 2>&1

In [2]:
# @title Importing Variables and Packages

import os
import json
from typing import List, Dict, Any, Optional

import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

from google.colab import userdata
# userdata.get('OPENAI_API_KEY')

# TinyTroupe imports
import tinytroupe
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe.extraction import ResultsExtractor as extractor # Reverted to original import

# Optional: Gradio UI
try:
    import gradio as gr
    HAS_GRADIO = True
except:
    HAS_GRADIO = False

# print("TinyTroupe version:", tinytroupe.__version__)
print("Gradio available:", HAS_GRADIO)

# Require ONLY the OpenAI API key
api_key = userdata.get('OPENAI_API_KEY')

if not api_key:
    raise RuntimeError(
        "Missing OPENAI_API_KEY.\n" # Fixed SyntaxError here
        "Set it before running this notebook:\n\n"
        "Windows PowerShell:\n"
        "   setx OPENAI_API_KEY \"your_key\"\n\n"
        "macOS/Linux:\n"
        "   export OPENAI_API_KEY=\"your_key\""
    )
# Ensure the API key is set as an environment variable for TinyTroupe to pick up
os.environ['OPENAI_API_KEY'] = api_key

print("OPENAI_API_KEY is set \u2713")


!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!

Looking for default config on: /usr/local/lib/python3.12/dist-packages/tinytroupe/utils/../config.ini
Failed to find custom config on: /content/config.ini
Will use only default values. IF THINGS FAIL, TRY CUSTOMIZING MODEL, API TYPE, etc.
TinyTroupe version: 0.5.2
Current date and time (local): 2025-11-19 20:20:21
Current date and time (UTC):   2025-11-19 20:20:21

Current TinyTroupe configuration 
[OpenAI]
api_type = openai
azure_api_version = 2023-05-15
model = gpt-4.1-mini
reasoning_model = o3-mini
embedding_model = text-embedding-3-small
max_tokens = 32000
temperature = 1.5
freq_penalty = 0.1
presence_penalty = 0.1
timeout = 480
max_attempts = 5
waiting_time = 1
exponential_backoff_factor = 5
reasoning_effort 

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Gradio available: True
OPENAI_API_KEY is set ✓


In [3]:
# ===============================================
# @title 2. PERSONA DEFINITIONS
# ===============================================

PersonaConfig = Dict[str, Any]

predefined_personas: Dict[str, PersonaConfig] = {

    "busy_parent": {
        "name": "Aisha Rahman",
        "description": """
        You are a 38-year-old working mother with two kids in Queens, NYC.
        You juggle childcare, commuting, and household duties.
        You are tech-competent but not advanced.
        You get frustrated by confusing flows or anything requiring lots of attention.
        """,
        "usage_context": """
        You mostly use apps while multitasking—on the subway, cooking, or late at night.
        You skim content and rely on simplicity.
        """,
        "goals": """
        Save time, reduce stress, avoid mistakes, keep your kids safe.
        """,
    },

    "college_student": {
        "name": "Miguel Santos",
        "description": """
        You are a 21-year-old CS undergrad.
        Tech-savvy, curious, heavy laptop & mobile user.
        You enjoy customization, dark mode, and efficiency.
        """,
        "usage_context": """
        You multitask heavily between school apps, Discord, coding, and browsing.
        """,
        "goals": """
        Speed, control, convenience, integrations with your workflow.
        """,
    },

    "privacy_conscious_user": {
        "name": "Jordan Kim",
        "description": """
        You are a 32-year-old privacy researcher.
        You deeply care about data rights, surveillance, and dark patterns.
        You scrutinize any data usage carefully.
        """,
        "usage_context": """
        You use VPNs, privacy tools, and carefully read permissions.
        """,
        "goals": """
        Transparency, user control, clear defaults, no hidden tracking.
        """,
    },

    "older_non_technical": {
        "name": "Linda Thompson",
        "description": """
        You are a 67-year-old retired teacher.
        Technology makes you anxious because you fear making mistakes.
        You prefer simple, clear flows and large text.
        """,
        "usage_context": """
        Mostly using a tablet at home in the evenings.
        Small text and complex menus overwhelm you.
        """,
        "goals": """
        Simplicity, reassurance, clear instructions, plain language.
        """,
    },
}

print("Loaded personas:", list(predefined_personas.keys()))

Loaded personas: ['busy_parent', 'college_student', 'privacy_conscious_user', 'older_non_technical']


In [4]:
# ===============================================
# @title 3. Create TinyPerson objects from persona configs
# ===============================================

def create_tiny_person_from_config(config: PersonaConfig) -> TinyPerson:
    person = TinyPerson(config["name"])

    person.define("persona_background", config["description"])
    person.define("usage_context", config["usage_context"])
    person.define("goals_and_priorities", config["goals"])

    person.define(
        "evaluation_mindset",
        """
        You are evaluating a digital feature.
        React like a real human with your background.
        Be detailed: likes, concerns, confusion, trust issues, accessibility problems,
        and emotional reactions.
        Stay in character.
        """
    )

    return person

# Clear existing agents before testing to avoid "Agent name already in use" error
# This is crucial if the cell is run multiple times in the same session.
if hasattr(tinytroupe.agent.TinyPerson, 'all_agents'):
    tinytroupe.agent.TinyPerson.all_agents.clear()

# Test create
_ = create_tiny_person_from_config(predefined_personas["busy_parent"])
print("TinyPerson factory OK ✓")


TinyPerson factory OK ✓


In [5]:
# ===============================================
# @title 4. Build simulation world + run conversation
# ===============================================

def build_world_for_feature(
    persona_keys: List[str],
    feature_description: str,
    world_name: str = "Feature Simulation"
) -> TinyWorld:

    # Clear existing agents before creating new ones to avoid 'Agent name already in use' error
    if hasattr(tinytroupe.agent.TinyPerson, 'all_agents'):
        tinytroupe.agent.TinyPerson.all_agents.clear()

    # Clear existing environments before creating a new one to avoid 'Environment name already in use' error
    if hasattr(tinytroupe.environment.TinyWorld, 'all_environments'):
        tinytroupe.environment.TinyWorld.all_environments.clear()

    persons = [
        create_tiny_person_from_config(predefined_personas[k])
        for k in persona_keys
    ]

    world = TinyWorld(world_name, persons)

    world.broadcast(f"""
    A new feature is being evaluated.

    FEATURE:
    {feature_description}

    Instructions:
    - React exactly as your persona would.
    - Share what confuses you or concerns you.
    - Mention emotional reactions.
    - Feel free to disagree with others.
    - Provide suggestions.
    """)

    return world


def run_feature_simulation(
    persona_keys: List[str],
    feature_description: str,
    steps: int = 3,
    extraction_objective: Optional[str] = None,
    verbose_extraction: bool = False,
):

    if extraction_objective is None:
        extraction_objective = """
        Summarize each persona's perspective:
        - Positive reactions
        - Concerns + risks
        - Confusing parts
        - Accessibility issues
        - Trust/privacy issues
        - Suggestions for improvement
        - Likelihood of adoption
        """

    world = build_world_for_feature(persona_keys, feature_description)
    world.run(steps)

    results_extractor_instance = extractor()
    report = results_extractor_instance.extract_results_from_world(
        tinyworld = world,
        extraction_objective = extraction_objective,
        verbose=verbose_extraction,
    )

    return {"world": world, "report": report}


In [6]:
# ===============================================
# @title 5. FORMAT REPORT INTO MARKDOWN
# ===============================================

def format_extracted_report(report: Any) -> str:
    if isinstance(report, dict):
        lines = ["# Persona Simulation Summary\n"]
        for key, section in report.items():
            lines.append(f"## {key}")
            lines.append("```json")
            lines.append(json.dumps(section, indent=2, ensure_ascii=False))
            lines.append("```")
        return "\n".join(lines)

    try:
        return "```json\n" + json.dumps(report, indent=2) + "\n```"
    except:
        return str(report)


In [7]:
# ===============================================
# @title 6. SIMPLE EXAMPLE TEST RUN
# ===============================================

feature = """
A new Smart Checkout page with:
- Single-page flow
- Auto-selected 'recommended shipping'
- Automatic discount code application
- Collapsible 'how your data is used' section
"""

personas = ["busy_parent", "college_student", "privacy_conscious_user"]
# personas = ["busy_parent"]

result = run_feature_simulation(personas, feature, steps=2)
formatted = format_extracted_report(result["report"])
print(formatted[:2500])


USER --> Aisha Rahman: [CONVERSATION] 
          >      A new feature is being evaluated.      FEATURE:      A new Smart Checkout page
          > with: - Single-page flow - Auto-selected 'recommended shipping' - Automatic discount
          > code application - Collapsible 'how your data is used' section       Instructions:     -
          > React exactly as your persona would.     - Share what confuses you or concerns you.
          > - Mention emotional reactions.     - Feel free to disagree with others.     - Provide
          > suggestions.

       + --> Miguel Santos

       + --> Jordan Kim

───────────────────────────────────────── Feature Simulation step 1 of 2 ──────────────────────────────────────────

Jordan Kim acts: [THINK] 
                > I'm evaluating the new Smart Checkout page feature. I notice it has a single-page
                > flow, which could be convenient, but I worry about the auto-selected 'recommended
                > shipping' option. This might pressure users into choices they don't want. The
                > automatic discount code application is nice, but I wonder if users are informed
                > about it transparently. The collapsible 'how your data is used' section concerns
                > me because hiding privacy information by default can reduce transparency and user
                > control. Overall, I feel cautious and somewhat skeptical about potential dark
                > patterns here.

...

Miguel Santos --> Aisha Rahman: [CONVERSATION] 
                   > I totally get the concerns about the auto-selected shipping feeling like a dark
                   > pattern. It kinda takes away control, which bugs me too. And hiding the data
                   > usage info behind a collapsible section? Not cool. Privacy stuff should be
                   > upfront and clear, not something you have to dig for. I really like the single-
                   > page flow and automatic discount codes for making things faster, but trust is a
                   > big deal here. They should definitely make the data usage section fully visible
                   > by default and let users easily opt out of any tracking or data sharing. That
                   > would make me feel way more comfortable using it.

       + --> Jordan Kim

Jordan Kim --> Aisha Rahman: [CONVERSATION] 
                > I agree with both Aisha and Miguel. The auto-selected shipping option could easily
                > lead to users getting stuck with choices they didn't actively make, which is
                > frustrating and undermines user control. The collapsible 'how your data is used'
                > section should definitely be more visible and transparent to avoid users
                > overlooking important privacy information. Miguel's idea of a quick summary
                > showing the selected shipping and applied discount is excellent for building trust
                > and clarity. Overall, the feature should prioritize transparency and user
                > empowerment over mere convenience.

       + --> Miguel Santos

# Persona Simulation Summary

## Aisha Rahman
```json
{
  "Positive reactions": [
    "Likes the single-page flow for simplicity",
    "Appreciates automatic discount code application for saving time and money"
  ],
  "Concerns + risks": [
    "Worries about auto-selected shipping causing mistakes if not noticed",
    "Uneasy about privacy due to collapsible 'how your data is used' section being easy to miss",
    "Concerned about feeling pushed into unwanted choices especially when multitasking or rushing"
  ],
  "Confusing parts": [
    "Potential confusion or second-guessing due to auto-selected options",
    "Collapsible data usage section might be overlooked"
  ],
  "Accessibility issues": [
    "Implied difficulty in noticing important info when busy or multitasking"
  ],
  "Trust/privacy issues": [
    "Privacy concerns from hidden data usage info",
    "Wants more visible and clear explanation of data usage to build trust"
  ],
  "Suggestions for improvement": [
    "Make data 

In [8]:
# ===============================================
# @title 7. BACKEND WRAPPER FOR UI
# ===============================================

def simulate_feature_for_ui(
    feature_description: str,
    persona_keys: List[str],
    steps: int = 3,
):
    result = run_feature_simulation(
        persona_keys=persona_keys,
        feature_description=feature_description,
        steps=steps,
    )

    summary_md = format_extracted_report(result["report"])

    return {
        "markdown_summary": summary_md,
        "raw_report": result["report"],
    }

print("UI wrapper ready ✓")


UI wrapper ready ✓


In [10]:
# ===============================================
# @title 8. GRADIO UI (optional)
# ===============================================

if not HAS_GRADIO:
    print("Install gradio with: pip install gradio")
else:
    persona_choices = list(predefined_personas.keys())

    def gradio_sim(
        feature_desc,
        persona_keys,
        steps,
        followup,
        history_json
    ):
        try:
            history = json.loads(history_json) if history_json else []
        except:
            history = []

        full_feature = feature_desc
        if followup.strip():
            full_feature += "\n\nFOLLOW-UP QUESTION:\n" + followup.strip()

        out = simulate_feature_for_ui(full_feature, persona_keys, steps)
        summary = out["markdown_summary"]

        history.append({
            "feature": feature_desc,
            "followup": followup,
            "personas": persona_keys,
            "summary_markdown": summary,
            "raw_report": out["raw_report"],
        })

        new_hist = json.dumps(history, indent=2, ensure_ascii=False)
        return summary, new_hist


    with gr.Blocks() as demo:
        gr.Markdown("""
        # 🎭 Persona Simulation Beta App
        **Powered by TinyTroupe + OpenAI**

        This tool lets you test how different types of users ("personas") would
        react to a new product feature.
        Provide a feature description, pick personas, and get simulated feedback.
        """)

        # -------------------------------
        # Feature Description
        # -------------------------------
        gr.Markdown("""
        ### 📝 Feature Description
        **What to enter here:**
        Describe the feature you want to test.
        Include enough detail so personas understand what the feature does,
        how users interact with it, and why it exists.

        **Examples:**
        - "A redesigned checkout page that auto-applies coupon codes."
        - "A new dark mode that activates automatically at sunset."
        - "An AI assistant that summarizes long emails into bullet points."
        """)

        feature_box = gr.Textbox(
            label="Describe your feature",
            lines=8,
            placeholder="Example: A new single-page checkout flow that automatically selects the recommended shipping option..."
        )

        # -------------------------------
        # Persona selection
        # -------------------------------
        gr.Markdown("""
        ### 👥 Personas
        **What this means:**
        Choose which types of users will evaluate the feature.

        Each persona has its own background, goals, frustrations, and behavior.

        **Select at least one.**
        """)

        persona_box = gr.CheckboxGroup(
            label="Select personas",
            choices=persona_choices,
            value=["busy_parent", "college_student"],
        )

        # -------------------------------
        # Follow-up question
        # -------------------------------
        gr.Markdown("""
        ### ❓ Follow-Up Question (Optional)
        **What to enter here:**
        Ask the personas a specific question after their initial reaction.

        **Examples:**
        - "What part is confusing?"
        - "Would this reduce or increase your trust in the product?"
        - "How likely are you to use this feature daily?"
        """)

        followup_box = gr.Textbox(
            label="Optional follow-up question",
            lines=3,
            placeholder="Example: What concerns you most about this feature?"
        )

        # -------------------------------
        # Steps slider
        # -------------------------------
        gr.Markdown("""
        ### 🔄 Simulation Steps
        Higher steps = deeper, more detailed conversations between personas.
        **Recommended:** 3–4 for typical use.
        """)

        steps_slider = gr.Slider(
            minimum=1,
            maximum=6,
            value=3,
            step=1,
            label="Number of conversation turns"
        )

        # Hidden history
        history_state = gr.Textbox(
            visible=False,
            label="Internal History JSON"
        )

        # Output
        output_md = gr.Markdown(label="Persona Feedback")

        # Button
        simulate_button = gr.Button("Run Simulation")

        simulate_button.click(
            fn=gradio_sim,
            inputs=[feature_box, persona_box, steps_slider, followup_box, history_state],
            outputs=[output_md, history_state]
        )

    demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cf379c93bb34ccb45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
# ===============================================
# @title 6. Test Run
# ===============================================

feature = """
web page leave confirmation when trying to leave on a checkout page:
- asks user to confirm that they want to leave checkout page
- provides option to return to previous page
- provides user to confirm that they want to leave
- provides user option to cancel and stay
- provides user option to return to more shopping page
"""

personas = ["busy_parent", "college_student", "privacy_conscious_user"]
# personas = ["busy_parent"]

result = run_feature_simulation(personas, feature, steps=2)
formatted = format_extracted_report(result["report"])
print(formatted[:2500])


USER --> Aisha Rahman: [CONVERSATION] 
          >      A new feature is being evaluated.      FEATURE:      web page leave confirmation
          > when trying to leave on a checkout page: - asks user to confirm that they want to leave
          > checkout page - provides option to return to previous page - provides user to confirm
          > that they want to leave - provides user option to cancel and stay - provides user option
          > to return to more shopping page       Instructions:     - React exactly as your persona
          > would.     - Share what confuses you or concerns you.     - Mention emotional reactions.
          > - Feel free to disagree with others.     - Provide suggestions.

       + --> Miguel Santos

       + --> Jordan Kim

───────────────────────────────────────── Feature Simulation step 1 of 2 ──────────────────────────────────────────

Miguel Santos acts: [THINK] 
                   > Okay, so this new feature is about a confirmation prompt when leaving a
                   > checkout page. I like the idea of preventing accidental navigation away during
                   > checkout, which can be frustrating. But I wonder about the user experience:
                   > will this confirmation annoy users who intentionally want to leave? Also, the
                   > options seem a bit redundant—'cancel and stay' and 'return to previous page'
                   > might overlap. I also think about how this integrates with browser behavior and
                   > if it respects accessibility standards. Emotionally, I'm a bit skeptical
                   > because I hate interruptions when I'm in a flow, but I get the safety aspect.
                   > Maybe it could be customizable or only trigger under certain conditions.

...

Miguel Santos --> Aisha Rahman: [CONVERSATION] 
                   > I totally get the concerns about this leave confirmation feature. Too many
                   > options can definitely overwhelm users, especially when they're in a rush or
                   > multitasking. Clear, simple wording like 'Yes, leave' and 'No, stay' sounds way
                   > better to me. Also, having the pop-up remember your choice for a while would
                   > save a lot of hassle and prevent annoyance. Plus, it's crucial that the feature
                   > doesn't feel like it's trapping users or making them hesitate unnecessarily.
                   > Overall, it should protect against accidental exits without messing up the
                   > checkout flow or stressing people out.

       + --> Jordan Kim

Jordan Kim --> Aisha Rahman: [CONVERSATION] 
                > I agree with both Aisha and Miguel. The leave confirmation should definitely be
                > simple and clear, using straightforward language like 'Yes, leave' and 'No, stay'
                > to avoid confusion. It should also be smart enough to only appear when there's a
                > real risk of losing data, like when the cart isn't empty or the user has entered
                > information. Remembering the user's choice temporarily would reduce annoyance.
                > Overall, the feature must respect user autonomy and not feel like a trap or an
                > obstacle, especially on mobile devices or when users are in a hurry.

       + --> Miguel Santos

# Persona Simulation Summary

## Aisha Rahman
```json
{
  "Positive reactions": "Appreciates the intent to prevent accidental leaving and losing the cart.",
  "Concerns + risks": "Finds extra pop-ups annoying and interruptive, especially when multitasking or in a rush; multiple options may be confusing or overwhelming; worries about clarity of options and how it works on mobile or in distracted environments; fears added stress instead of reducing it.",
  "Confusing parts": "Multiple options like 'confirm leaving', 'cancel and stay', 'return to shopping' might confuse users; unclear wording could cause second-guessing and wrong clicks.",
  "Accessibility issues": "Concerned about usability on mobile devices and in situations with limited attention (e.g., subway).",
  "Trust/privacy issues": "Worries about feeling pressured or tricked into staying on the page.",
  "Suggestions for improvement": "Simplify to a clear yes/no question with simple phrases like 'Yes, leave' and 'No, stay'; hav